<a href="https://www.kaggle.com/code/amartinez111/house-price-predictor?scriptVersionId=114444773" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# !pip install -U tensorboard witwidget keract
!pip install ipython

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory-
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import time
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (logs/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.layers import Dense, Flatten, Input
# from keract import get_activations, display_activations
import scipy as sp

from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.ticker as ticker
import matplotlib.patches as patches

from IPython.display import HTML


from math import sin, cos, pi
import math
import re # regex expresions

#  first of all, i create animation figure to see training stats

In [ ]:
class AnimationFigure():
    def __init__(self, frames, shape, show_weights):
        self.fig = plt.figure(figsize=(14, 11), constrained_layout=True)
        # grid = fig.add_gridspec(2, 2, hspace=0.25,wspace=0.25)
        self.grid = self.fig.add_gridspec(1, 2)
        # grid.update(left=0.6,right=0.5,top=0.965,bottom=0.03)
        self.frames = frames
        
        self.gridGeneralFunction = self.grid[0].subgridspec(2, 1)
        self.axPredict = self.fig.add_subplot(self.gridGeneralFunction[0])
        self.axJFunction = self.fig.add_subplot(self.gridGeneralFunction[1])
        
        self.gridParametersChange = self.grid[1].subgridspec(shape[0] * shape[1] + 1, 1) 
        
        self.wShape = shape
        
        self.wAxis = np.zeros(shape, dtype=object)
        self.bAxis = np.zeros((1), dtype=object)
        
        self.bAxis.itemset((0), self.fig.add_subplot(self.gridParametersChange[shape[0] * shape[1]]))
        
        if show_weights:
            for f in range(0, shape[0]):
                for c in range(0, shape[1]):
                    self.wAxis.itemset((f, c), self.fig.add_subplot(self.gridParametersChange[shape[1] * f + c]))

    def init_func(self, x_data, y_data, call, show_weights):
        
        # print(x_data[:,0])
        # print(y_data)
        # print(call(x_data))
        
        self.points = self.axPredict.scatter(x_data[:,0], y_data, color="blue", label="Dataset values")       
        self.pointsPred = self.axPredict.scatter(x_data[:,0], call(x_data), color="None", lw=2, label='Predicted values')
        
        self.parametersChangeW = np.zeros((self.frames, self.wShape[0], self.wShape[1]), dtype=np.float64)
        self.parametersChangeB = np.zeros((self.frames, 1, 1), dtype=np.float64)

        self.axJFunction.set_title('Cost function')
        self.axJFunction.set_ylabel('J(w,b)')
        self.axJFunction.set_xlabel('frames')
        self.axJFunction.axis("auto")
        self.axJFunction.margins(.1, .1)

        self.axPredict.set_title('P w,b (size)')
        self.axPredict.set_ylabel('House pricing (Tomans)')
        self.axPredict.set_xlabel('Area (m^2)')
        self.axPredict.axis("auto")
        self.axPredict.set_ylim(bottom=0)

        if show_weights:
            for f in range(0, self.wShape[0]):
                for c in range(0, self.wShape[1]):
                    self.wAxis.item((f, c)).axis("auto")
                    self.setup(self.wAxis.item((f, c)), "w{}{} parameter".format(f, c))

            self.bAxis.item((0)).axis("auto")
            self.setup(self.bAxis.item((0)), "b parameter")
    
        return self.pointsPred,
        # return self.pointsPred, self.parametersChangeW, self.parametersChangeB
                        
    # Setup a plot such that only the bottom spine is shown
    def setup(self, ax, title):
        ax.spines['right'].set_color('none')
        ax.spines['left'].set_color('none')
        ax.spines['top'].set_color('none')

        ax.yaxis.set_major_locator(ticker.NullLocator())
        ax.xaxis.set_ticks_position('bottom')

        ax.tick_params(which='major', width=1.00)
        ax.tick_params(which='major', length=5)
        ax.tick_params(which='minor', width=0.75)
        ax.tick_params(which='minor', length=2.5)

        ax.set_xlim(auto=True)
        ax.set_ylim(-.8, .8)

        ax.patch.set_alpha(0.0)
        ax.xaxis.set_major_locator(ticker.AutoLocator())
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
        ax.set_title(title, fontsize=14)
        
    def onChangeParameter(self, f, field, col, parameter, parametersChange, ax):
        xf = parameter
        # print(xf)
        
        parametersChange.itemset((f, field, col), xf)
        
        if f >= 1:
            xi = parametersChange.item((f - 1, field, col))
            rad = 0.2 # round((4 + (xf-xi) ** 2) / 8)
            point = ax.scatter(xi, 0)
            
            arrow = patches.FancyArrowPatch(
                (xi, 0),
                (xf, 0),
                arrowstyle="->",
                mutation_scale=25,
                connectionstyle="arc3,rad={}".format(rad)
            )
            
            ax.add_patch(arrow)
            point.set_offsets((xf, 0))
            
        if f > 4:
            lastChanges = parametersChange[f-3: f + 1, field, col]
            
            minimum = np.amin(lastChanges)
            maximum = np.amax(lastChanges)
                        
            rate = 0.015
            
            # this function works better than others
            ax.set_xlim(minimum * (1 - rate) - rate, maximum * (1 + rate) + rate)
        

    def update(self, f, fit, x_data, y_data, call, model, show_weights):        
        print('Frame: {}'.format(str(f)))
        
        # get info
        history = fit()
        
        w = model.get_weights()[0].item((0, 0))
        b = model.get_weights()[1].item((0))
        
        y_pred = call(x_data)
        
        j_value_key = list(history.history.keys())[1]
        j_eval_key = list(history.history.keys())[2]
            
        
        error_value = round(history.history[j_value_key][0], 2)
        val_error_value = round(history.history[j_eval_key][0], 2)
        
        # plot this 
        self.axJFunction.scatter(f, error_value, color='k', lw=2)
        self.axJFunction.scatter(f, val_error_value, color='r', lw=2)

        self.axJFunction.text(
            self.axJFunction.axis()[1] * 1.02,
            error_value * 1.02,
            str(error_value),
            horizontalalignment='right',
            verticalalignment='center',
        )
        
        self.pointsPred.set_offsets(
           np.hstack((
                x_data[:,0].reshape((len(y_pred), 1)),
                y_pred.reshape((len(y_pred), 1))
            ))
        )
        self.pointsPred.set_color("red")
        
        # line = self.axPredict.scatter(x_data[:,0], y_pred, color='k', lw=2, label='Predicted values')
        
        if show_weights:
            W = model.get_weights()[0]
            shape = W.shape
            for row in range(0, shape[0]):
                for col in range(0, shape[1]):
                    self.onChangeParameter(f, row, col, W.item((row, col)), self.parametersChangeW, self.wAxis.item((row, col)))

            self.onChangeParameter(f, 0, 0, b, self.parametersChangeB, self.bAxis.item((0)))

        self.axPredict.set_title("P w,b (size) Frame: {}".format(
            str(f)
        ))

        return self.pointsPred,
#         return [self.pointsPred, self.parametersChangeW, self.parametersChangeB]

# then i will create line model that uses the model

In [ ]:
from sklearn.model_selection import train_test_split

class LineModel(tf.keras.Model):
    def __init__(self,
            x_data = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 20.0], dtype=np.float32),
            y_data = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 20.0], dtype=np.float32),
            weights = None,
            model = None,
            # inputFunction = lambda x: np.array([x], dtype=np.float32),
            inputFunction = lambda x: x,
            outputFunction = lambda y: y,
            frames = 12,
            animation = True,
            id = int(time.time()),
            show_weights = True
        ):        
        super().__init__()

        self.x_data = x_data
        self.y_data =  y_data
        self.id = id
        self.model = model
        self.inputFunction = inputFunction
        self.outputFunction = outputFunction
        self.frames = frames
        self.show_weights = show_weights
        self.model.summary()
        self.dirCheckpoint = "logs/line-model-{}".format(self.id)
        self.fileCheckpoint = self.dirCheckpoint+"/line-model.h5"
        self.history = dict()
        
        self.tensorboard_callback = tf.keras.callbacks.TensorBoard(
            self.dirCheckpoint,  histogram_freq=1
        )
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.x_data, self.y_data, test_size=0.33, shuffle=True)

        if animation:
            self.plotFunction()
        else:
            self.fit(epochs=frames)

        self.model.save(self.fileCheckpoint)

    
    def plotFunction(self):
        animationFigure = AnimationFigure(self.frames, self.model.get_weights()[0].shape, self.show_weights)

        self.ani = animation.FuncAnimation(
            animationFigure.fig,
            animationFigure.update,
            self.frames,
            lambda: animationFigure.init_func(self.x_data, self.y_data, self.call, self.show_weights),
            fargs=(self.fit, self.x_data, self.y_data, self.call, self.model, self.show_weights),
            interval=340,
            blit=True,
            repeat=False
        )

        # print("save")
        self.ani.save("model-{}.mp4".format(self.id))
        # print("end save")
        # HTML(linear_model.ani.to_html5_video())

    def fit(self, epochs=1):        
        local_history = self.model.fit(
            self.inputFunction(self.X_train),
            self.outputFunction(self.y_train),
            batch_size=20,
            epochs=epochs,
            validation_data=(
                self.inputFunction(self.X_test),
                self.outputFunction(self.y_test),
            )
            # callbacks=[self.tensorboard_callback]
        )
        
        for some_key in local_history.history.keys():
            if some_key not in self.history:
                self.history[some_key] = local_history.history[some_key]
            else:
                self.history[some_key] += local_history.history[some_key]

        return local_history
    
    def call(self, inputs):
        # print(self.model.predict(self.inputFunction(inputs)))
        return self.model.predict(self.inputFunction(inputs))
    
    def print_model(self):
        return tf.keras.utils.plot_model(self.model, show_shapes=True)

In [ ]:
from IPython.display import Video

# Now i will get house pricing dataset csv with pandas
### and i will analyze the data frame

In [ ]:
house_price_Iran_Df = pd.read_csv("../input/house-price-tehran-iran/housePrice.csv")

In [ ]:
df = house_price_Iran_Df.loc[:,['Area', 'Room', 'Parking', 'Warehouse', 'Elevator', 'Address', 'Price', 'Price(USD)' ]]

In [ ]:
df.describe()

In [ ]:
print(df.sample(5)) # head
df.head(5)

In [ ]:
print((df.Price / df["Price(USD)"]).median())
print((df.Price / df["Price(USD)"]).mean())
tomansToDollarsScale = (df.Price / df["Price(USD)"]).mean()

In [ ]:
print(df.info())

# Data preprocessing
##### i will try to convert all these data types to the reasonable type it should contain
* ### area should be int64 but it be an object

In [ ]:
try:
    df.Area = df.Area.astype('int64')
except Exception as e:
    print(e)

In [ ]:
df.query('Area==\' 3,310,000,000 \'')

In [ ]:
try: 
    df.Area = df.Area.apply(lambda x: re.sub(',', '', x))
    df.Area = pd.to_numeric(df.Area)
except TypeError as e:
    if str(e) == 'expected string or bytes-like object':
        print('Area type is already a number')
    else:
        print(e)
except Exception as e:
    print(e)

* ### i will convert all addresses to objects

In [ ]:
try:
    df.Address = df.Address.astype('string')
except Exception as e:
    print(e)

* ### i will convert these boolean values to 0 and 1

In [ ]:
df["Parking"] = df["Parking"].astype(int)
df["Elevator"] = df["Elevator"].astype(int)
df["Warehouse"] = df["Warehouse"].astype(int)

In [ ]:
print(df.info())

# I will trimmed out unusual features values.

In [ ]:
df.plot.scatter(x = "Area", y = "Price")

upperLimit = 0.90
lowerLimit = 1 - upperLimit

areaTrimmed = df["Area"].quantile([upperLimit, lowerLimit])
priceTrimmed = df["Price"].quantile([upperLimit, lowerLimit])
priceInDollarsTrimmed = df["Price(USD)"].quantile([upperLimit, lowerLimit])

# print(df)

df.Area = df.Area.drop(df[
    (df.Area > areaTrimmed[upperLimit]) |
    (df.Area < areaTrimmed[lowerLimit])
].index)

df.Price = df.Price.drop(df[
    (df.Price > priceTrimmed[upperLimit]) |
    (df.Price < priceTrimmed[lowerLimit])
].index)

df["Price(USD)"] = df["Price(USD)"].drop(df[
    (df["Price(USD)"] > priceInDollarsTrimmed[upperLimit]) |
    (df["Price(USD)"] < priceInDollarsTrimmed[lowerLimit])
].index)

print(df)

df = df.dropna()

print(df)
df.plot.scatter(x = "Area", y = "Price")


# Feature engineering
### i need to scale features that i use for increase model performance, avoid high output variation when i do a weight shift. For now only gonna to take areaList and pricingList as output and do feature scaling

In [ ]:
def scaleFeature(feature_name, dfAux, extra_scale):
    dfLocal = dfAux.loc[:,[feature_name]]

    scale = int(dfLocal.max()) / extra_scale
    df_norm = dfLocal / scale
    
    dfAux = dfAux.drop([feature_name], axis=1)
    dfAux = pd.concat((df_norm, dfAux), 1)
    
    return dfAux, scale
 
df, priceScale = scaleFeature("Price", df, 8/12)
df, priceUSDScale = scaleFeature("Price(USD)", df, 1)
df, areaScale = scaleFeature("Area", df, 8/10)
df, roomScale = scaleFeature("Room", df, 1)

# print(df.query('Area < 2/3'))
df.plot.scatter(x = "Area", y = "Price")

In [ ]:
df.plot.scatter(x = "Area", y = "Price")
df.sample(5)

In [ ]:
# house_price_Iran_Df = df.loc[0:99]
house_price_Iran_Df = df

area = house_price_Iran_Df["Area"].to_numpy()
price = house_price_Iran_Df["Price"].to_numpy()

area = area.reshape((len(area), 1))

# print(area)
# print(area[:,0])
print(area.shape)
print(price.shape)
# print(price)
print(house_price_Iran_Df.sample(5))

# Run linear model and display its results

In [ ]:
model=Sequential([
    Input(shape=(1, 1)),
    # Flatten(),
    Dense(units=1, activation="linear", name="funcion-{}".format("001"))
])
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.009), # 0.04
    loss='mean_absolute_error',
    metrics=['mean_absolute_error'],
)
# model.fit(
#    area,
#    price,
#    epochs=8,
#    batch_size=20,
#)

for i in range(3):
        
    house_price_model = LineModel(
        # house_price_Iran_Df["Area"].values,
        id="house_price_model1",
        x_data=area,
        y_data=price,
        model=model,
        weights=[
            np.array([[-1.0]], dtype=np.float32),
            np.array([2.0], dtype=np.float32)
        ],
        frames = 12,  # 5
        animation = True,
        show_weights = True,
    )
    
    loss = house_price_model.history["loss"]
    
    if abs(loss[-1] - loss[0]) > 0.001:
        break


Video("model-{}.mp4".format(house_price_model.id))

# i will try to modify the model
### I will do the model twice accept the area of the house as input

In [ ]:
print(house_price_Iran_Df["Area"].values)
print(house_price_Iran_Df["Room"].values)
print(house_price_Iran_Df.loc[:, ["Area", "Room"]].values)

In [ ]:
def inputFunction(x):
    return x

def outputFunction(y):
    y_transformed = np.matrix(y).transpose()
    return y_transformed

model = Sequential([
    Input(shape=(1, 2)),
    Dense(10, activation="relu"),
    Dense(4, activation="relu",),
    Dense(1, activation="relu", name="funcion-a"),
    # Dense(1, name='result')
])
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.009), # 0.04
    loss='mean_absolute_error',
    metrics=['mean_absolute_error'],
)


for i in range(3):
    house_price_model_2 = LineModel(
        id="house_price_model_2",
        x_data=np.array(house_price_Iran_Df.loc[:, ["Area", "Room"]].values),
        y_data=house_price_Iran_Df["Price"].values,
        model = model,
        inputFunction = inputFunction,
        outputFunction = outputFunction,
        frames=12,
        animation=True,
        show_weights=False,
    )

    
    loss = house_price_model.history["loss"]
    
    if abs(loss[-1] - loss[0]) > 0.001:
        break
        
        
Video("model-{}.mp4".format(house_price_model_2.id))

# i will add Parking, Warehouse and Elevator boolean features

In [ ]:
def inputFunction(x):
    return x

def outputFunction(y):
    y_transformed = np.matrix(y).transpose()
    return y_transformed

model = Sequential([
    Input(shape=(1, 5)),
    Dense(1, activation="linear", name="funcion-a"),
    Dense(1, name='result')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.009), # 0.04
    loss='mean_absolute_error',
    metrics=['mean_absolute_error'],
)

house_price_model_3 = LineModel(
    id="house_price_model_3",
    x_data=np.array(house_price_Iran_Df.loc[:, ["Area", "Parking", "Room", "Warehouse", "Elevator"]].values),
    y_data=house_price_Iran_Df["Price"].values,
    model = model,
    # inputFunction = lambda x : np.array([x, x], dtype=np.float32)
    inputFunction = inputFunction,
    outputFunction = outputFunction,
    frames=19,
    animation=True,
    show_weights=True,
)
Video("model-{}.mp4".format(house_price_model_3.id))

# Model test

In [ ]:
w = house_price_model.model.get_weights()[0].item((0, 0))
b = house_price_model.model.get_weights()[1].item((0))

print("{} * x' + {} = y'".format(w, b))
print("y' = y / {}".format(10 * priceScale))
print("x' = x / {}".format(10 * areaScale))
print()

print("then,")
print("({} * x / {}) + {} = y / {}".format(w, 10*areaScale, b, 10*priceScale))
print()

print("then,")
print("{} * x + {} = {} * y".format(w / (10 * areaScale), b , 1 /(10 * priceScale)))

print("then,")
print("{} * x + {} = {} * y".format((10 * priceScale) * w / (10 * areaScale), (10 * priceScale) * b , 1))

In [ ]:
print("{} * areaScale + {} = pricingHouseInUSD".format(
    (priceScale * w) / (areaScale * tomansToDollarsScale),
    (b * priceScale) / tomansToDollarsScale
))

# First approximation house price prediction with area size alone
### the price of the house rises 2180 USD (+- 140) for each increase in area up to 1 m**2

### 1 USD = 30,000 Tomans (in extra info at house price dataset).

# I will try with an example

In [ ]:
df = house_price_Iran_Df
df["Area"] = df["Area"] * areaScale
df["Room"] = df["Room"] * roomScale
df["Price"] = df["Price"] * priceScale
df["Price(USD)"] = df["Price(USD)"] * priceUSDScale

# Scale list variable
* priceScale
* priceUSDScale
* areaScale
* roomScale

In [ ]:
example = df.loc[30:60]
print(example)

In [ ]:
example["Area"] = example["Area"] / areaScale
example["Room"] = example["Room"] / roomScale
example["Price"] = example["Price"] / priceScale
example["Price(USD)"] = example["Price(USD)"] / priceUSDScale

In [ ]:
print(example)

# example error

In [ ]:
print("house_price_model", np.mean(house_price_model.call(np.array([
    example.Area
]).reshape((-1, 1)))).mean() * priceScale / tomansToDollarsScale)

print("house_price_model_2", house_price_model_2.call(np.array([
    example.Area, example.Room
]).reshape(-1, 2)).mean() * priceScale / tomansToDollarsScale)

print("house_price_model_3", house_price_model_3.call(np.array([
    example.Area, example.Parking, example.Room, example.Warehouse, example.Elevator
]).reshape((-1,5))).mean() * priceScale / tomansToDollarsScale)


In [ ]:
print("house_price_model", np.mean(house_price_model.call(np.array([
    example.Area
]).reshape((-1, 1))).reshape((-1)) - example.Price) * priceScale / tomansToDollarsScale)

print("house_price_model_2", np.mean(house_price_model_2.call(np.array([
    example.Area, example.Room
]).reshape((-1, 2))).reshape((-1)) - example.Price) * priceScale / tomansToDollarsScale)

print("house_price_model_3", np.mean(house_price_model_3.call(np.array([
    example.Area, example.Parking, example.Room, example.Warehouse, example.Elevator
]).reshape((-1,5))).reshape((-1)) - example.Price)  * priceScale / tomansToDollarsScale)

### In this example, the model 1 and 2 are better than the 3th model.

<h1> I will predict house price with other dataset with more information</h1>

<p>/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv</p>
<p>/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt</p>
<p>/kaggle/input/house-prices-advanced-regression-techniques/train.csv</p>
<p>/kaggle/input/house-prices-advanced-regression-techniques/test.cs</p>

In [ ]:
dataset_advanced_regression = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
len(dataset_advanced_regression)

In [ ]:
dataset_advanced_regression.sample(5)

In [ ]:
dataset_advanced_regression.info()

<h2> i will verificate types </h2>

In [ ]:
dataset_advanced_regression.loc[:,["LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope", "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle"]].value_counts(dropna=False)

In [ ]:
dataset_advanced_regression.loc[:,["RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType"]].value_counts(dropna=False)

<h2> I will do not delete None values because the None value can introduce some value of the training model </h2> 

In [ ]:
dataset_advanced_regression.loc[:,["ExterQual", "ExterCond", "Foundation", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2"]].value_counts(dropna=False)

In [ ]:
dataset_advanced_regression.loc[:,["GarageYrBlt", "GarageFinish"]].value_counts(dropna=False)

<h2> How we can see these are cathegorical variables and verify if i need to add this or put zero to this features or leave to stay null</h2> 

In [ ]:
print(dataset_advanced_regression.query("LotFrontage==0").loc[:,"LotFrontage"])
print(dataset_advanced_regression.query("MasVnrArea==0").loc[:,"MasVnrArea"])
print(dataset_advanced_regression.query("GarageYrBlt==0").loc[:,"GarageYrBlt"])

In [ ]:
dataset_advanced_regression_only_numbers = pd.get_dummies(dataset_advanced_regression, dummy_na=True)

In [ ]:
dataset_advanced_regression_only_numbers.info(verbose=True, null_counts=True) # [939]

In [ ]:
dataset_advanced_regression_only_numbers["LotFrontage_nan"] = dataset_advanced_regression_only_numbers.loc[:,"LotFrontage"].isnull().astype(int)
dataset_advanced_regression_only_numbers["MasVnrArea_nan"] = dataset_advanced_regression_only_numbers.loc[:,"MasVnrArea"].isnull().astype(int)

print(dataset_advanced_regression_only_numbers.loc[:,["LotFrontage","LotFrontage_nan", "MasVnrArea", "MasVnrArea_nan"]].sample(10))


In [ ]:
dataset_advanced_regression_only_numbers["LotFrontage"] = dataset_advanced_regression_only_numbers.loc[:,"LotFrontage"].fillna(dataset_advanced_regression_only_numbers.loc[:,"LotFrontage"].mean()) # mean doesnt affect the general prediction
dataset_advanced_regression_only_numbers["MasVnrArea"] = dataset_advanced_regression_only_numbers.loc[:,"MasVnrArea"].fillna(dataset_advanced_regression_only_numbers.loc[:,"MasVnrArea"].mean())
# dataset_advanced_regression_only_numbers["GarageYrBlt"] = dataset_advanced_regression_only_numbers.loc[:,"GarageYrBlt"].fillna(dataset_advanced_regression_only_numbers.loc[:,"GarageYrBlt"].mean())
# 

In [ ]:
dataset_advanced_regression["Alley"].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
# .loc[:,["BsmtUnfSF", "KitchenAbvGr"]]

sns.heatmap(dataset_advanced_regression_only_numbers.iloc[:,np.r_[20:30, 37, 333, 334]].corr(), ax=ax, annot=True, cmap=sns.cubehelix_palette(as_cmap=True))

<h2> i will use LotFrontage, LotArea, LotFrontage_nan and LotArea_nan to predict SalePrice because exists a correlation between SalePrice and these features </h2>

In [ ]:
first_database_implementation = dataset_advanced_regression_only_numbers.loc[:, [
    "LotArea",
    "MSSubClass",
    "LotFrontage",
    "MasVnrArea",
    "MasVnrArea_nan",
    "LotFrontage_nan",
    "YearBuilt",
    "YearRemodAdd",
    "BsmtFinSF1",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "GrLivArea",
    "BsmtFullBath",
    "FullBath",
    "SalePrice"
]]

In [ ]:
print(first_database_implementation.info())
first_database_implementation.sample(5)

In [ ]:
first_database_implementation["SquareLotArea"] = first_database_implementation["LotArea"] ** 2
first_database_implementation["LotAreaDotLotFrontage"] = first_database_implementation["LotArea"] * first_database_implementation["LotFrontage"]

In [ ]:
first_database_implementation = first_database_implementation[[
    "LotArea",
    "SquareLotArea","LotAreaDotLotFrontage", # 
    "MSSubClass",
    "LotFrontage",
    "MasVnrArea_nan",
    "LotFrontage_nan",
    "YearBuilt",
    "YearRemodAdd",
    "BsmtFinSF1",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "GrLivArea",
    "BsmtFullBath",
    "FullBath",
    "SalePrice"
]]

# I will trimmed out unusual features values again.

In [ ]:
upper_limit = 0.90
lower_limit = 1 - upperLimit

lot_area_trim = first_database_implementation["LotArea"].quantile([upperLimit, lowerLimit])
square_lot_area_trim = first_database_implementation["SquareLotArea"].quantile([upperLimit, lowerLimit])
lot_frontage_trim = first_database_implementation["LotFrontage"].quantile([upperLimit, lowerLimit])

def cut_dataframe(df, feature, trim_list):
    df[feature] = df[feature].drop(df[
        (df[feature] > trim_list[upper_limit]) |
        (df[feature] < trim_list[lower_limit])
    ].index)
    return df

first_database_implementation = cut_dataframe(first_database_implementation, "LotArea", lot_area_trim)
first_database_implementation = cut_dataframe(first_database_implementation, "SquareLotArea", square_lot_area_trim)
first_database_implementation = cut_dataframe(first_database_implementation, "LotFrontage", lot_frontage_trim)

first_database_implementation = first_database_implementation.dropna()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X=first_database_implementation.values[:,0:-1]
y=first_database_implementation.values[:,-1]

print(X)
print(f"shape: {X.shape}")
print()
print(y)
print(f"shape: {y.shape}")


<h2> I will turn left and then will normalize by row and will turn right </h2>
<p> i do that because sklearn doesnt allow to normalize by column </p>

In [ ]:
# qt = preprocessing.QuantileTransformer(random_state=0)
# qt.fit_transform(X)
min_max_scaler = preprocessing.MinMaxScaler()
X_transformed = min_max_scaler.fit_transform(X)

transform_normalize = preprocessing.Normalizer(norm="max")
y_transformed = transform_normalize.fit_transform(y.reshape(1, -1)).reshape((-1, 1))


# X_transformed = np.transpose(X)

# transform_normalize = preprocessing.Normalizer(norm="l2")
# X_transformed = transform_normalize.transform(X_transformed)

# X_transformed = np.transpose(X_transformed)

# y_transformed = transform_normalize.transform(y.reshape(1,-1)).reshape((-1))

print(X_transformed[0:10])
print(X_transformed.shape)

print(y_transformed[0:10])
print(y_transformed.shape)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y_transformed, test_size=0.20, shuffle=True)

In [ ]:
X_test

In [ ]:
def run(model, X, y, X_test,  y_test):
    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.03),
        loss=tf.keras.losses.MeanAbsoluteError(),
        metrics=[tf.keras.metrics.MeanSquaredError()]
    )
    model.save_weights('aux_model.h5')
    
    for i in range(3):        
        model.summary()
        model.load_weights('aux_model.h5')
        model.fit(
            X,
            y,
            epochs=12,
            batch_size=35,
            validation_data=(X_test,  y_test),
        )
        loss = model.history.history["loss"]

        if abs(loss[-1] - loss[0]) > 0.001:
            break

    return model

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(1, 2)),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1, activation="relu"),
    # tf.keras.layers.Dense(1, activation="relu"),
])

model_trained = run(model, X_train[:,0:2], y_train, X_test[:,0:2],  y_test)


In [ ]:
model_with_four_variables = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(1, 16)),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1, activation="relu"),
    # tf.keras.layers.Dense(1, activation="relu"),
])
run(model_with_four_variables, X_train, y_train, X_test,  y_test)

## For plot model result, i will compact all dimensions in only LotArea and SalePrice parameters, i will see the shade of the function
#### i was decide to hide weights change graphs because it is not possible to read what is the algorithm doing and is slowly

In [ ]:
identity_initializer = tf.keras.initializers.Identity()

model_with_four_variables = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(1, 16)),
    tf.keras.layers.Dense(10, activation="relu"),    
    tf.keras.layers.Dense(4, activation="relu"),
    tf.keras.layers.Dense(1, activation="relu"),
])

model_with_four_variables.compile(
    #optimizer=tf.keras.optimizers.Adam(),
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.06),
    loss=tf.keras.losses.MeanAbsoluteError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)

for i in range(3):
    house_price_model_advanced = LineModel(
        id="house_price_advanced",
        x_data=X_train,
        y_data=y_train,
        model = model_with_four_variables,
        # inputFunction = inputFunction,
        # outputFunction = outputFunction,
        frames=10,
        animation=True,
        show_weights=False,
    )
    loss = house_price_model_advanced.history["loss"]
    
    if abs(loss[-1] - loss[0]) > 0.001:
        break


Video("model-{}.mp4".format(house_price_model_advanced.id))

## It doesnt mean that the algorithm works well
### This plot show us that the shade of house_price_model_advanced is the same shade of the training set getting only two parameters of all parameters, house area and price.
## Then, how we can verify that the model function represents the training set?
### We will try to get others parameters and see that the shade is the same that the shade of the model in that dimensions, i will do math ortogonal projection of the function to the two features plane all that n times.
(Or we can create an image that randomly change the ) 

### Note: First of all we would like to see that the model works well in the training set, later we will see if the model works with data that doesnt have seen before.

In [ ]:
def get_plot(model, feature_idx, X_train, y_train, columns_dataframe, color_predicted, label_dataset_value):
    
    plt.scatter(X_train[:,feature_idx], y_train, color="blue", label=label_dataset_value)
    plt.scatter(X_train[:,feature_idx], model.predict(X_train), color=color_predicted, lw=2, label='Predicted values')
    
    plt.xlabel(columns_dataframe[feature_idx])
    plt.ylabel('House price')
    plt.legend(loc="upper left")

    plt.show()
    
for feature_idx in range(0, X.shape[1] - 1):
    get_plot(
        model_with_four_variables,
        feature_idx,
        X_train,
        y_train,
        first_database_implementation.columns[0:-1],
        color_predicted="green",
        label_dataset_value="Training values"
    )
    
    # the next plot is red because this plot show us that the model can generalize well or not
    get_plot(
        model_with_four_variables,
        feature_idx,
        X_train,
        y_train,
        first_database_implementation.columns[0:-1],
        color_predicted="red",
        label_dataset_value="Test values"
    )


In [ ]:
import IPython.display as ipd


beep = np.sin(1*np.pi*400*np.arange(10000*1)/10000)
ipd.Audio(beep, rate=10000, autoplay=True)